In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install librosa

In [3]:
import numpy as np
import pandas as pd
import librosa
from scipy.io import wavfile

In [4]:
import torch
import torch.nn as nn
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Processor, AdamW, get_linear_schedule_with_warmup

In [5]:
# 데이터 로드
df_all_txt = pd.read_csv("/content/drive/MyDrive/Text_Audio_Multimodal/data/df_calculation.csv")

In [6]:
all_wav_tmp = df_all_txt["wav_id"].tolist()
all_emotion = df_all_txt["label"].tolist()

In [7]:
# Wav2Vec2 Processor 및 모델 로드
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
model = Wav2Vec2ForSequenceClassification.from_pretrained("facebook/wav2vec2-base", num_labels=4)

model.to('cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2ForSequenceClassification(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)


In [8]:
# audio data -> wav2vec input
def audio2input(filepath):
    samplerate, audio = wavfile.read(filepath, mmap=True)
    if samplerate != 16000:  # Check if the sampling rate is different from 16000 Hz
        audio = librosa.resample(audio.astype(np.float32), orig_sr=samplerate, target_sr=16000)  # Resample to 16000 Hz if necessary
    input_values = processor(audio, sampling_rate=16000, return_tensors="pt").input_values  # Explicitly set sampling rate
    return input_values

In [9]:
import time

docs = []
total_files = len(all_wav_tmp)

start_time = time.time()  # 시작 시간 기록

for i in range(total_files):
    filename = all_wav_tmp[i]
    label = all_emotion[i]

    input_values = audio2input(f"/content/drive/MyDrive/Text_Audio_Multimodal/data/merged_wav_folder/{filename}.wav")
    input_values = input_values.to('cuda')

    docs.append({
        'fileName': filename,
        'input_values': input_values,
        'label': torch.tensor([int(label)], device='cuda')  # Convert label to integer
    })

    # 진행 상황 출력 (100개마다)
    if (i + 1) % 100 == 0 or (i + 1) == total_files:
        print(f"Processed {i+1}/{total_files} files...")

# 최종적으로 걸린 시간 출력
end_time = time.time()
elapsed_time = end_time - start_time
print(f"All files have been processed in {elapsed_time:.2f} seconds.")


Processed 100/15463 files...
Processed 200/15463 files...
Processed 300/15463 files...
Processed 400/15463 files...
Processed 500/15463 files...
Processed 600/15463 files...
Processed 700/15463 files...
Processed 800/15463 files...
Processed 900/15463 files...
Processed 1000/15463 files...
Processed 1100/15463 files...
Processed 1200/15463 files...
Processed 1300/15463 files...
Processed 1400/15463 files...
Processed 1500/15463 files...
Processed 1600/15463 files...
Processed 1700/15463 files...
Processed 1800/15463 files...
Processed 1900/15463 files...
Processed 2000/15463 files...
Processed 2100/15463 files...
Processed 2200/15463 files...
Processed 2300/15463 files...
Processed 2400/15463 files...
Processed 2500/15463 files...
Processed 2600/15463 files...
Processed 2700/15463 files...
Processed 2800/15463 files...
Processed 2900/15463 files...
Processed 3000/15463 files...
Processed 3100/15463 files...
Processed 3200/15463 files...
Processed 3300/15463 files...
Processed 3400/1546

In [10]:
# # # output_file.txt에서 파일명을 읽어와서 리스트로 저장 (output_file.txt)
# # with open('/content/drive/MyDrive/Text_Audio_Multimodal/output_file.txt', 'r') as f:
# #     wav_files_to_load = [line.strip().replace('.wav', '') for line in f.readlines()]

# # output_file.txt에서 파일명을 읽어와서 리스트로 저장 (filtered_output_file.txt -> )
# with open('/content/drive/MyDrive/Text_Audio_Multimodal/filtered_output_file.txt', 'r') as f:
#     wav_files_to_load = [line.strip() for line in f.readlines()]

# print(wav_files_to_load)

In [11]:
# import time

# docs = []
# total_files = len(wav_files_to_load)

# start_time = time.time()

# for i in range(total_files):
#     filename = all_wav_tmp[i]
#     label = all_emotion[i]

#     # output_file.txt에 있는 파일만 처리
#     if filename in wav_files_to_load:
#         input_values = audio2input(f"/content/drive/MyDrive/Text_Audio_Multimodal/data/merged_wav_folder/{filename}.wav")
#         input_values = input_values.to('cuda')

#         docs.append({
#             'fileName': filename,
#             'input_values': input_values,
#             'label': torch.tensor([int(label)], device='cuda')  # Convert label to integer
#         })

#         # 진행 상황 출력
#         if (i + 1) % 100 == 0 or (i + 1) == total_files:
#             print(f"Processed {i+1}/{total_files} files...")

# # 최종적으로 걸린 시간 출력
# end_time = time.time()
# elapsed_time = end_time - start_time
# print(f"All selected files have been processed in {elapsed_time:.2f} seconds.")


In [18]:
NUM_EPOCHS = 10

In [25]:
# Train / Test Split
# data split(total 15463) -> 12370(0.8)+3093(0.2)
train_list = docs[:12370]
test_list = docs[12370:]

criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=2e-4, eps=1e-8)

total_steps = len(train_list) * NUM_EPOCHS

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

total_steps = 1

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [27]:
import time
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score

# 손실 및 정확도를 저장할 리스트 초기화
train_losses = []
train_accuracies = []
train_f1_scores = []
total_steps = 0

# 학습
for epoch in range(NUM_EPOCHS):
    model.train()

    # 에포크 시작 시간 기록
    start_time = time.time()

    epoch_loss = 0.0
    all_preds = []
    all_labels = []

    for every_trainlist in train_list:
        input_values = every_trainlist['input_values']
        label = every_trainlist['label']

        optimizer.zero_grad()
        outputs = model(input_values, labels=label)
        loss = outputs.loss

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        epoch_loss += loss.item()

        # 예측 및 라벨 저장
        preds = torch.argmax(outputs.logits, dim=-1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(label.cpu().numpy())

        total_steps += 1

    # 에포크 종료 시간 기록 및 경과 시간 계산
    end_time = time.time()
    elapsed_time = end_time - start_time

    # 경과 시간을 시/분/초로 변환하여 출력
    hours, rem = divmod(elapsed_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print(f'Epoch {epoch + 1} completed in {int(hours)}h {int(minutes)}m {int(seconds)}s')

    # 에포크별 평균 손실 계산
    avg_loss = epoch_loss / len(train_list)
    train_losses.append(avg_loss)

    # 에포크별 정확도 계산
    accuracy = accuracy_score(all_labels, all_preds)
    train_accuracies.append(accuracy)

    # F1 스코어 계산
    f1 = f1_score(all_labels, all_preds, average='weighted')  # 'weighted' or 'macro' based on your needs
    train_f1_scores.append(f1)

    print(f'Epoch {epoch + 1} Training Loss: {avg_loss:.4f} Training Accuracy: {accuracy:.4f} F1 Score: {f1:.4f}')

Epoch 1 completed in 0h 14m 45s
Epoch 1 Training Loss: 1.4136 Training Accuracy: 0.4152 F1 Score: 0.3181
Epoch 2 completed in 0h 14m 46s
Epoch 2 Training Loss: 1.4061 Training Accuracy: 0.4150 F1 Score: 0.3178
Epoch 3 completed in 0h 14m 42s
Epoch 3 Training Loss: 1.3988 Training Accuracy: 0.4157 F1 Score: 0.3174
Epoch 4 completed in 0h 14m 27s
Epoch 4 Training Loss: 1.3968 Training Accuracy: 0.4135 F1 Score: 0.3142
Epoch 5 completed in 0h 14m 31s
Epoch 5 Training Loss: 1.3960 Training Accuracy: 0.4145 F1 Score: 0.3124
Epoch 6 completed in 0h 14m 29s
Epoch 6 Training Loss: 1.3965 Training Accuracy: 0.4123 F1 Score: 0.3079
Epoch 7 completed in 0h 14m 30s
Epoch 7 Training Loss: 1.3966 Training Accuracy: 0.4106 F1 Score: 0.2889
Epoch 8 completed in 0h 14m 27s
Epoch 8 Training Loss: 1.3896 Training Accuracy: 0.4130 F1 Score: 0.2905
Epoch 9 completed in 0h 14m 28s
Epoch 9 Training Loss: 1.3812 Training Accuracy: 0.4116 F1 Score: 0.2880
Epoch 10 completed in 0h 14m 28s
Epoch 10 Training Loss

In [29]:
# 학습 모델 저장
torch.save(model, './audio_train_model.pt')

RuntimeError: Serialization of parametrized modules is only supported through state_dict(). See:
https://pytorch.org/tutorials/beginner/saving_loading_models.html#saving-loading-a-general-checkpoint-for-inference-and-or-resuming-training

In [ ]:
# train 된 결과 불러오기
model = torch.load("audio_train_model.pt")
model.eval()
model.to('cuda')

In [ ]:
y_actu = []
y_pred = []
output_list = []

In [ ]:
# 테스트
for every_test_list in test_list:
    label1 = every_test_list['label']
    label1 = torch.tensor([label1])
    input_values = every_test_list['input_values']

    with torch.no_grad():
        outputs = model(input_values)
        logits = outputs.logits

        _, preds = torch.max(logits, dim=1)
        y_actu.append(label1.numpy()[0])
        y_pred.append(preds.cpu().numpy()[0])

        out = logits.detach().cpu().numpy()
        output_list.append(out)

In [ ]:
# 결과 저장
output_np = np.array(output_list).reshape(len(test_list), 7)
np.save("./audio_output.npy", output_np)
np.save("./audio_truth.npy", y_actu)
np.save("./audio_pred.npy", y_pred)

In [ ]:
# 결과 출력
from sklearn.metrics import f1_score, accuracy_score

f1score = f1_score(y_actu, y_pred, average="weighted")
print("f1 socre :", f1score)

acc = accuracy_score(y_actu, y_pred)
print("acc :", acc)

In [ ]:
import time

while True:
    time.sleep(120)  # 120초마다 한 번씩 메시지를 출력
    print("Colab session is active.")
